In [1]:
from import_src import import_src
import_src()

Append /home/v_user/khiempm/MLOpsVN/src to sys.path


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import os
import json
from plotly import express as px
import mlflow

In [30]:
pd.set_option('display.max_columns', 500)

In [4]:
link_data_test_raw = '../../data/captured_data/phase-2/prob-2/raw/'
train_raw_data = '../../data/raw_data/phase-2/prob-2/raw_train.parquet'

In [7]:
raw_test = pd.DataFrame()
count_ = 0
for file_path in os.listdir(link_data_test_raw):
    if 'parquet' not in file_path or "123.parquet" in file_path:
        continue
    captured_data = pd.read_parquet(f'{link_data_test_raw}{file_path}')
    captured_data['parquet_idx'] = count_
    raw_test = pd.concat([captured_x, captured_data])
    count_ += 1

In [40]:
raw_test['feature7'].describe()

count    1.044440e+05
mean     9.185845e+03
std      1.602658e+05
min      4.600000e+01
25%      1.140000e+02
50%      5.340000e+02
75%      1.470000e+03
max      1.018725e+07
Name: feature7, dtype: float64

In [41]:
raw_train['feature7'].describe()

count    6.184100e+04
mean     8.226342e+03
std      1.692162e+05
min      2.400000e+01
25%      1.140000e+02
50%      5.280000e+02
75%      1.392000e+03
max      1.435577e+07
Name: feature7, dtype: float64

In [23]:
order_cols = raw_train.columns.tolist()
order_cols.remove('label')
raw_test = raw_test[order_cols]

In [27]:
raw_test['feature1'].describe()

count    104444.000000
mean          1.296438
std           6.141171
min           0.000000
25%           0.000008
50%           0.004351
75%           0.672780
max          59.999943
Name: feature1, dtype: float64

In [28]:
raw_train['feature1'].describe()

count    61841.000000
mean         1.250073
std          6.007840
min          0.000000
25%          0.000008
50%          0.004414
75%          0.685279
max         59.999935
Name: feature1, dtype: float64

In [9]:
raw_train = pd.read_parquet(train_raw_data)

In [35]:
raw_train['feature1'].value_counts()

feature1
0.000009    6800
0.000008    3763
0.000003    3763
0.000005    2255
0.000004    1996
            ... 
0.185814       1
1.068945       1
1.084268       1
0.105288       1
0.929489       1
Name: count, Length: 29954, dtype: int64